In [2]:
#행정동 법정동 매핑 데이터프레임 불러오기

import pandas as pd

mapping_bh_dong = pd.read_csv('clean_map_dh_dong.csv', index_col=0)
mapping_bh_dong.head()

,시군구명,행정동코드,행정동명,법정동코드,법정동명,법정동개수
0,종로구,11110515,청운효자동,11110101,청운동,10
1,종로구,11110515,청운효자동,11110102,신교동,10
2,종로구,11110515,청운효자동,11110103,궁정동,10
3,종로구,11110515,청운효자동,11110104,효자동,10
4,종로구,11110515,청운효자동,11110105,창성동,10


In [16]:

# yearmonth = ['201911','202005','202009','202101','202106','202110','202201','202204']
yearmonth = ['201911','202005','202009']

for i in yearmonth:

    live_df = pd.read_csv('거주인구_'+i+'.csv', encoding = 'euc-kr')
# live_df = pd.read_csv('거주인구_202005.csv', encoding = 'euc-kr')

    # 행정구역 중 '서울특별시' 제거
    # 연령 중 '계' 제거
    # 컬럼명 변경
    # 인덱스,단위,불필요한 컬럼 제거
    live_df = live_df[(live_df['행정구역(동읍면)별']!='서울특별시')&(live_df['5세별']!='계')& (live_df['항목']!='총인구수[명]')]
    live_df = live_df.reset_index()
    live_df = live_df.rename(columns = {'행정구역(동읍면)별':'행정동', '5세별':'연령대','항목':'성별', i[:4]+'.'+i[-2:]+' 월':'거주인구수'})
    live_df = live_df.drop(['index','단위','Unnamed: 5'], axis = 1)


    # 연령대: 5세기준 -> 10세 기준
    # 성별 표준화: oo인구수[명] -> oo
    live_df = live_df.replace(['0 - 4세','5 - 9세'],'10대 미만', regex = True)
    live_df = live_df.replace(['10 - 14세', '15 - 19세'],'10대', regex = True)
    live_df = live_df.replace(['20 - 24세', '25 - 29세'],'20대', regex = True)
    live_df = live_df.replace(['30 - 34세', '35 - 39세'],'30대', regex = True)
    live_df = live_df.replace(['40 - 44세', '45 - 49세'],'40대', regex = True)
    live_df = live_df.replace(['50 - 54세', '55 - 59세'],'50대', regex = True)
    live_df = live_df.replace(['60 - 64세', '65 - 69세'],'60대', regex = True)
    live_df = live_df.replace(['70 - 74세', '75 - 79세','80 - 84세', '85 - 89세', '90 - 94세', '95 - 99세', '100+'],'70대 이상', regex = True)
    live_df.loc[live_df['연령대']=='70대 이상+','연령대'] ='70대 이상'

    live_df['성별'] = live_df['성별'].apply(lambda x: x[:2])


    #행정동 내 구로 되어 있는 행 삭제
    ## 행정동 내 value가 지역구인 것은 지역구 컬럼에 작성
    ## 나머지는 앞의 값에 따라 채우는 걸로
    ## 인덱스 삭제
    gu = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
            '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
            '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']

    live_df.loc[live_df['행정동'].isin(gu),'시군구명'] = live_df.loc [live_df['행정동'].isin(gu),'행정동']
    live_df = live_df.fillna(method = "ffill")
    live_df = live_df[live_df['행정동'].isin(gu) == False].reset_index()
    live_df = live_df.drop('index', axis = 1)

    display(live_df)


    #지역, 행정동, 성별, 연령별로 인구수 sum
    live_df = live_df[['시군구명','행정동','연령대','성별','거주인구수']].groupby(['시군구명','행정동','성별','연령대']).sum().reset_index()


    #유동인구데이터프레임과 행정동법정동매핑데이터 합치기
    live_df_b = pd.merge(live_df,mapping_bh_dong)[['시군구명','법정동코드','법정동명','법정동개수','성별','연령대','거주인구수']]

    live_df_b = live_df_b.groupby(['시군구명','법정동코드','법정동명','성별','연령대','법정동개수']).sum().reset_index()

    # 거주인구를 법정동 수로 나누기 
    live_df_b['거주인구수/n'] = live_df_b['거주인구수']/live_df_b['법정동개수']
    live_df_b = live_df_b[['시군구명','법정동코드','법정동명','연령대','거주인구수/n']]

    live_df_b.to_csv('cleanlivedf'+i+'.csv')


,행정동,연령대,성별,거주인구수,시군구명
0,청운효자동,10대 미만,남자,195,종로구
1,청운효자동,10대 미만,여자,193,종로구
2,청운효자동,10대 미만,남자,264,종로구
3,청운효자동,10대 미만,여자,250,종로구
4,청운효자동,10대,남자,306,종로구
...,...,...,...,...,...
17803,둔촌제2동,70대 이상,여자,66,강동구
17804,둔촌제2동,70대 이상,남자,2,강동구
17805,둔촌제2동,70대 이상,여자,23,강동구
17806,둔촌제2동,70대 이상,남자,1,강동구


,행정동,연령대,성별,거주인구수,시군구명
0,청운효자동,10대 미만,남자,191,종로구
1,청운효자동,10대 미만,여자,178,종로구
2,청운효자동,10대 미만,남자,274,종로구
3,청운효자동,10대 미만,여자,243,종로구
4,청운효자동,10대,남자,288,종로구
...,...,...,...,...,...
17845,둔촌제2동,70대 이상,여자,71,강동구
17846,둔촌제2동,70대 이상,남자,3,강동구
17847,둔촌제2동,70대 이상,여자,17,강동구
17848,둔촌제2동,70대 이상,남자,1,강동구


,행정동,연령대,성별,거주인구수,시군구명
0,청운효자동,10대 미만,남자,177,종로구
1,청운효자동,10대 미만,여자,168,종로구
2,청운효자동,10대 미만,남자,271,종로구
3,청운효자동,10대 미만,여자,227,종로구
4,청운효자동,10대,남자,275,종로구
...,...,...,...,...,...
17845,둔촌제2동,70대 이상,여자,75,강동구
17846,둔촌제2동,70대 이상,남자,4,강동구
17847,둔촌제2동,70대 이상,여자,17,강동구
17848,둔촌제2동,70대 이상,남자,1,강동구


In [15]:

# yearmonth = ['201911','202005','202009','202101','202106','202110','202201','202204']
# yearmonth = ['201911','202005','202009']
yearmonth = ['202101','202106','202110','202201','202204']

for i in yearmonth:

    live_df = pd.read_csv('거주인구_'+i+'.csv', encoding = 'euc-kr')
# live_df = pd.read_csv('거주인구_202005.csv', encoding = 'euc-kr')

    # 행정구역 중 '서울특별시' 제거
    # 연령 중 '계' 제거
    # 컬럼명 변경
    # 인덱스,단위,불필요한 컬럼 제거
    live_df = live_df[(live_df['행정구역(동읍면)별']!='서울특별시')&(live_df['5세별']!='계')& (live_df['항목']!='총인구수[명]')]
    live_df = live_df.reset_index()
    live_df = live_df.rename(columns = {'행정구역(동읍면)별':'행정동', '5세별':'연령대','항목':'성별', i[:4]+'.'+i[-2:]+' 월':'거주인구수'})
    live_df = live_df.drop(['index','[A]행정구역(동읍면)별','[Item]항목','[B]5세별','단위','Unnamed: 8'], axis = 1)


    # 연령대: 5세기준 -> 10세 기준
    # 성별 표준화: oo인구수[명] -> oo
    live_df = live_df.replace(['0 - 4세','5 - 9세'],'10대 미만', regex = True)
    live_df = live_df.replace(['10 - 14세', '15 - 19세'],'10대', regex = True)
    live_df = live_df.replace(['20 - 24세', '25 - 29세'],'20대', regex = True)
    live_df = live_df.replace(['30 - 34세', '35 - 39세'],'30대', regex = True)
    live_df = live_df.replace(['40 - 44세', '45 - 49세'],'40대', regex = True)
    live_df = live_df.replace(['50 - 54세', '55 - 59세'],'50대', regex = True)
    live_df = live_df.replace(['60 - 64세', '65 - 69세'],'60대', regex = True)
    live_df = live_df.replace(['70 - 74세', '75 - 79세','80 - 84세', '85 - 89세', '90 - 94세', '95 - 99세', '100+'],'70대 이상', regex = True)
    live_df.loc[live_df['연령대']=='70대 이상+','연령대'] ='70대 이상'

    live_df['성별'] = live_df['성별'].apply(lambda x: x[:2])


    #행정동 내 구로 되어 있는 행 삭제
    ## 행정동 내 value가 지역구인 것은 지역구 컬럼에 작성
    ## 나머지는 앞의 값에 따라 채우는 걸로
    ## 인덱스 삭제
    gu = ['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
            '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
            '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']

    live_df.loc[live_df['행정동'].isin(gu),'시군구명'] = live_df.loc [live_df['행정동'].isin(gu),'행정동']
    live_df = live_df.fillna(method = "ffill")
    live_df = live_df[live_df['행정동'].isin(gu) == False].reset_index()
    live_df = live_df.drop('index', axis = 1)

    display(live_df)


    #지역, 행정동, 성별, 연령별로 인구수 sum
    live_df = live_df[['시군구명','행정동','연령대','성별','거주인구수']].groupby(['시군구명','행정동','성별','연령대']).sum().reset_index()


    #유동인구데이터프레임과 행정동법정동매핑데이터 합치기
    live_df_b = pd.merge(live_df,mapping_bh_dong)[['시군구명','법정동코드','법정동명','법정동개수','성별','연령대','거주인구수']]

    live_df_b = live_df_b.groupby(['시군구명','법정동코드','법정동명','성별','연령대','법정동개수']).sum().reset_index()

    # 거주인구를 법정동 수로 나누기 
    live_df_b['거주인구수/n'] = live_df_b['거주인구수']/live_df_b['법정동개수']
    live_df_b = live_df_b[['시군구명','법정동코드','법정동명','연령대','거주인구수/n']]

    live_df_b.to_csv('cleanlivedf'+i+'.csv')


,행정동,연령대,성별,거주인구수,시군구명
0,청운효자동,10대 미만,남자,164,종로구
1,청운효자동,10대 미만,여자,156,종로구
2,청운효자동,10대 미만,남자,266,종로구
3,청운효자동,10대 미만,여자,237,종로구
4,청운효자동,10대,남자,290,종로구
...,...,...,...,...,...
17845,둔촌제2동,70대 이상,여자,74,강동구
17846,둔촌제2동,70대 이상,남자,5,강동구
17847,둔촌제2동,70대 이상,여자,19,강동구
17848,둔촌제2동,70대 이상,남자,1,강동구


,행정동,연령대,성별,거주인구수,시군구명
0,청운효자동,10대 미만,남자,159,종로구
1,청운효자동,10대 미만,여자,152,종로구
2,청운효자동,10대 미만,남자,232,종로구
3,청운효자동,10대 미만,여자,230,종로구
4,청운효자동,10대,남자,300,종로구
...,...,...,...,...,...
17845,둔촌제2동,70대 이상,여자,67,강동구
17846,둔촌제2동,70대 이상,남자,6,강동구
17847,둔촌제2동,70대 이상,여자,23,강동구
17848,둔촌제2동,70대 이상,남자,1,강동구


,행정동,연령대,성별,거주인구수,시군구명
0,청운효자동,10대 미만,남자,147,종로구
1,청운효자동,10대 미만,여자,145,종로구
2,청운효자동,10대 미만,남자,239,종로구
3,청운효자동,10대 미만,여자,223,종로구
4,청운효자동,10대,남자,301,종로구
...,...,...,...,...,...
17887,둔촌제2동,70대 이상,여자,65,강동구
17888,둔촌제2동,70대 이상,남자,5,강동구
17889,둔촌제2동,70대 이상,여자,24,강동구
17890,둔촌제2동,70대 이상,남자,1,강동구


,행정동,연령대,성별,거주인구수,시군구명
0,청운효자동,10대 미만,남자,141,종로구
1,청운효자동,10대 미만,여자,130,종로구
2,청운효자동,10대 미만,남자,234,종로구
3,청운효자동,10대 미만,여자,220,종로구
4,청운효자동,10대,남자,301,종로구
...,...,...,...,...,...
17887,둔촌제2동,70대 이상,여자,67,강동구
17888,둔촌제2동,70대 이상,남자,6,강동구
17889,둔촌제2동,70대 이상,여자,17,강동구
17890,둔촌제2동,70대 이상,남자,0,강동구


,행정동,연령대,성별,거주인구수,시군구명
0,청운효자동,10대 미만,남자,131,종로구
1,청운효자동,10대 미만,여자,123,종로구
2,청운효자동,10대 미만,남자,231,종로구
3,청운효자동,10대 미만,여자,220,종로구
4,청운효자동,10대,남자,295,종로구
...,...,...,...,...,...
17887,둔촌제2동,70대 이상,여자,69,강동구
17888,둔촌제2동,70대 이상,남자,7,강동구
17889,둔촌제2동,70대 이상,여자,16,강동구
17890,둔촌제2동,70대 이상,남자,0,강동구
